# Technical Analysis Functions and New DB

In [1]:
import sqlite3
import pandas as pd
import numpy as np
import datetime
import plotly.graph_objects as go



In [2]:
# Get some stock prices from our Prices Database
# Load a list of ticker symbols to load
con1 = sqlite3.connect("TSX_Prices.sqlite")
#con2 = sqlite3.connect("TSX_Analysis.sqlite")

ticker = "SHOP"
sql = f"SELECT * FROM 'prices_daily' WHERE Ticker = '{ticker}' ORDER BY date ASC"
df = pd.read_sql_query(sql, con1)
df.drop(['index'], axis=1, inplace=True)
df["Date"] = pd.to_datetime(df["Date"], infer_datetime_format=True)
df.set_index("Date")
df.head()

,Date,Ticker,Open,High,Low,Close,Volume
0,2014-08-26,SHOP,-,-,-,0.04,0.0
1,2014-08-27,SHOP,-,-,-,0.04,0.0
2,2014-08-28,SHOP,-,-,-,0.04,0.0
3,2014-08-29,SHOP,-,-,-,0.04,0.0
4,2014-09-01,SHOP,-,-,-,0.04,0.0


In [3]:
df["GAP"] = df["Date"] - df["Date"].shift(1) 
df["PrevDate"] = df["Date"].shift(1)
df.loc[df["GAP"] > datetime.timedelta(days=3)]

,Date,Ticker,Open,High,Low,Close,Volume,GAP,PrevDate
289,2015-11-02,SHOP,42.36,43.99,40.52,40.52,51399.0,31 days,2015-10-02
333,2016-01-04,SHOP,35.48,36.0,34.15,35.92,36602.0,4 days,2015-12-31
363,2016-02-16,SHOP,28.96,29.64,27.64,28.48,266340.0,4 days,2016-02-12
391,2016-03-28,SHOP,36.68,36.68,35.52,36.37,40350.0,4 days,2016-03-24
431,2016-05-24,SHOP,34.53,35.82,34.49,35.57,70905.0,4 days,2016-05-20
480,2016-08-02,SHOP,44.74,45.02,42.94,43.93,69337.0,4 days,2016-07-29
504,2016-09-06,SHOP,55.54,55.95,54.74,55.58,57585.0,4 days,2016-09-02
528,2016-10-11,SHOP,58.95,58.95,55.67,56.04,151928.0,4 days,2016-10-07
582,2016-12-28,SHOP,58.5,60.0,58.18,58.72,144216.0,5 days,2016-12-23
585,2017-01-03,SHOP,58.22,58.4,56.6,57.65,83632.0,4 days,2016-12-30


In [4]:
df["previous_close"] = df["Close"].shift(1)
df["returns_manual"] = (df["Close"] / df["previous_close"]) - 1
df["change %"] = df["Close"] - df["previous_close"]
df["returns"] = df["Close"].pct_change()
# LN(1+return) -> EXP(sum of LN) - 1
df["log_return"] = np.log(1 + df["returns"])
df["cum_log"] = np.exp(df["log_return"].cumsum()) - 1
df["SMA200"] = df["Close"].rolling(window=250).mean()
df["Trend"] = np.where(df["Close"] > df["SMA200"], "Up", "Down")
df.tail(10)

,Date,Ticker,Open,High,Low,Close,Volume,GAP,PrevDate,previous_close,returns_manual,change %,returns,log_return,cum_log,SMA200,Trend
1798,2021-12-17,SHOP,1672.0,1731.71,1645.0,1705.37,447966.0,1 days,2021-12-16,1693.69,0.006896,11.68,0.006896,0.006873,42633.25,1662.01076,Up
1799,2021-12-20,SHOP,1693.54,1710.63,1649.58,1662.21,144553.0,3 days,2021-12-17,1705.37,-0.025308,-43.16,-0.025308,-0.025634,41554.25,1663.39124,Down
1800,2021-12-21,SHOP,1703.04,1770.87,1687.41,1770.16,154336.0,1 days,2021-12-20,1662.21,0.064944,107.95,0.064944,0.062922,44253.00,1665.12316,Up
1801,2021-12-22,SHOP,1764.83,1786.79,1745.0,1776.67,144806.0,1 days,2021-12-21,1770.16,0.003678,6.51,0.003678,0.003671,44415.75,1666.85784,Up
1802,2021-12-23,SHOP,1788.0,1847.9,1772.87,1840.00,138957.0,1 days,2021-12-22,1776.67,0.035645,63.33,0.035645,0.035025,45999.00,1668.62676,Up
1803,2021-12-24,SHOP,1828.06,1849.99,1812.37,1813.55,21461.0,1 days,2021-12-23,1840.00,-0.014375,-26.45,-0.014375,-0.014479,45337.75,1670.34552,Up
1804,2021-12-29,SHOP,1770.18,1791.14,1730.13,1781.47,162865.0,5 days,2021-12-24,1813.55,-0.017689,-32.08,-0.017689,-0.017847,44535.75,1671.88544,Up
1805,2021-12-30,SHOP,1776.31,1833.45,1776.31,1781.51,156278.0,1 days,2021-12-29,1781.47,0.000022,0.04,0.000022,0.000022,44536.75,1673.53220,Up
1806,2021-12-31,SHOP,1781.39,1781.4,1740.77,1741.69,98397.0,1 days,2021-12-30,1781.51,-0.022352,-39.82,-0.022352,-0.022605,43541.25,1675.11200,Up
1807,2022-01-04,SHOP,1742.37,1742.37,1540.0,1552.23,390884.0,4 days,2021-12-31,1741.69,-0.108779,-189.46,-0.108779,-0.115163,38804.75,1675.77796,Down


In [5]:
p1 = df.loc[df["Date"] == "2014-08-26"]["Close"].values[0]
p2 = df.loc[df["Date"] == "2021-12-31"]["Close"].values[0]
r1 = (p2 / p1) - 1
r1

43541.25

In [6]:
df_t = df.loc[df["Date"] >= "2021-05-01"]
df_t.set_index("Date")
df_t.tail(20)

,Date,Ticker,Open,High,Low,Close,Volume,GAP,PrevDate,previous_close,returns_manual,change %,returns,log_return,cum_log,SMA200,Trend
1788,2021-12-03,SHOP,1860.17,1875.71,1742.71,1809.67,223165.0,1 days,2021-12-02,1853.90,-0.023858,-44.23,-0.023858,-0.024147,45240.75,1639.05040,Up
1789,2021-12-06,SHOP,1767.44,1825.8,1719.58,1823.53,207095.0,3 days,2021-12-03,1809.67,0.007659,13.86,0.007659,0.007630,45587.25,1641.39336,Up
1790,2021-12-07,SHOP,1860.0,1943.0,1858.53,1924.47,230801.0,1 days,2021-12-06,1823.53,0.055354,100.94,0.055354,0.053876,48110.75,1644.23592,Up
1791,2021-12-08,SHOP,1897.99,1939.8,1885.79,1922.29,135841.0,1 days,2021-12-07,1924.47,-0.001133,-2.18,-0.001133,-0.001133,48056.25,1647.10456,Up
1792,2021-12-09,SHOP,1909.94,1945.0,1885.94,1899.12,112949.0,1 days,2021-12-08,1922.29,-0.012053,-23.17,-0.012053,-0.012127,47477.00,1649.88572,Up
1793,2021-12-10,SHOP,1908.09,1945.0,1835.97,1857.22,165525.0,1 days,2021-12-09,1899.12,-0.022063,-41.90,-0.022063,-0.022310,46429.50,1652.54372,Up
1794,2021-12-13,SHOP,1840.0,1880.27,1758.37,1792.15,288051.0,3 days,2021-12-10,1857.22,-0.035036,-65.07,-0.035036,-0.035665,44802.75,1654.83968,Up
1795,2021-12-14,SHOP,1762.44,1797.7,1709.91,1740.47,190801.0,1 days,2021-12-13,1792.15,-0.028837,-51.68,-0.028837,-0.029261,43510.75,1656.75356,Up
1796,2021-12-15,SHOP,1741.4,1765.08,1664.57,1753.59,367763.0,1 days,2021-12-14,1740.47,0.007538,13.12,0.007538,0.007510,43838.75,1658.58924,Up
1797,2021-12-16,SHOP,1815.21,1815.21,1660.86,1693.69,307372.0,1 days,2021-12-15,1753.59,-0.034158,-59.90,-0.034158,-0.034756,42341.25,1660.24280,Up


In [7]:
fig = go.Figure(data=[go.Candlestick(x=df_t['Date'],
                open=df_t['Open'],
                high=df_t['High'],
                low=df_t['Low'],
                close=df_t['Close'])
                ])

fig.add_trace(go.Scatter(x=df_t['Date'], 
                         y=df_t['SMA200'], 
                         opacity=0.7, 
                         line=dict(color='blue', width=4), 
                         name='SMA 200'))


fig.show()